# ROS Control MicroCourse

## Chapter 2: Configuring the controllers

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>1.5 hours</b><br><br>
This unit will explain what configurations you need to do in order to be able to use the ros_control package with your robot.

<p style="background:green;color:white;">END OF SUMMARY</p>

### Configuring the URDF (Transmissions)

In order to be able to use the ros_control package in your simulation, you first need to add some things to the URDF file that defines your robot. 

The first element you will need to define is the transmissions. The transmission elements are used to describe the relationship between a joint and an actuator. A transmission transforms efforts/flow variables such that their product - power - remains constant.

In order to better understand how these elements work, have a look at the next exercise and its subsequent explanation.

<p style="background:#EE9023;color:white;">Exercise 2.1</p>
<br>
a) Go to the rrbot_description package and copy the xacro file that defines the simulated robot in your workspace. To do that, just execute the following commands:
<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

b) You will now be able to visualize the files through the IDE, which is much more pleasant.

<img src="img/ide2.png" width="800" />

c) Have a look at the transmission tags that are defined in this file. You can find any word in the IDE by activating the "Find" tool, hitting Ctrl+F.

<p style="background:#EE9023;color:white;">End of Exercise 2.1</p>

### Explaining the file

As you have already seen in the file, the transmission tags have three main elements that you need to define:

<ul>
<li><b>&lt;type&gt;</b>: It defines the type of transmission that will be used. Currently, the only available option is <i><b>transmission_interface/SimpleTransmission</b></i>.</li> 
<br>
<img src="img/type_tag.png" width="600" />
<br>
<li><b>&lt;joint&gt;</b>: It defines the joint connected to this transmission. The name of the joint must match a joint that is already defined for the robot (in its URDF files). You also need to define the hardware interface it will use, in this case, the <i><b>EffortJointInterface</b></i>.</li>
<br>
<img src="img/joint_tag2.png" width="600" />
<br>
<li><b>&lt;actuator&gt;</b>: It defines the actuator connected to this transmission. Here you will define the hardware interface (as you did for the joint) and the mechanical reduction (this one is optional, but it is usually defined and set to 1).</li>
<br>
<img src="img/actuator_tag2.png" width="600" />
<br>
</ul>

### Configuring the URDF (Plugin)

Besides defining the transmission tags, you also need to add the gazebo ros control plugin to your URDF files. As we did before, follow the next exercise and its subsequent explanation in order to better understand how this works.

<p style="background:#EE9023;color:white;">Exercise 2.2</p>
<br>
a) Go to the rrbot_description package and copy the file that contains the Gazebo tags into your workspace. To do that, just execute the following commands:
<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

b) Have a look at the top Gazebo tag of this file and try to guess what it does.

<p style="background:#EE9023;color:white;">End of Exercise 2.2</p>

### Explaining the file

As you have seen, it looks quite simple. But... what exactly does this do? Let's see!

Basically, what this plugin does is read the information contained in the transmission tags you saw before and load it properly so that Gazebo can understand it. When loading this plugin, you will always have to add a plugin tag indicating the <i>name</i> of the plugin and the <i>filename</i> that contains it.

Inside this <i>&lt;plugin&gt;</i> tag, there are some tags that you can define:

<ul>
<li><b>&lt;robotNamespace&gt;</b>: The namespace to be used by the instance of this plugin. By default, it will put the name of the robot that's in the URDF file.</li>
<li><b>&lt;controlPeriod&gt;</b>: The period of the controller update (in seconds). By default, it uses the Gazebo period. This tag is usually left to its default value, so it's not defined.</li>
<li><b>&lt;robotParam&gt;</b>: The location of the robot_description parameter on the parameter server. By default, it uses '/robot_description'. This tag is usually left to its default value, so it's not defined.</li>
<li><b>&lt;robotSimType&gt;</b>: The pluginlib name of a custom robot hardware interface to be used. By default, it uses 'DefaultRobotHWSim'.</li>
</ul>

So, in most cases, you would just define the <i><b>&lt;robotNamespace&gt;</b></i> tag. But what about the <i><b>&lt;robotSimType&gt;</b></i> tag?

This tag will be used just in case you want to define a custom hardware interface, thus, for more advanced users. For what concerns this course, we are not going to see how to do this, so don't worry too much.

In the default behavior, which is either not setting this tag or setting it to 'DefaultRobotHWSim', you will be able to use the following interfaces, which are the ones you saw in the previous chapter:
<ul>
<li><b>JointStateInterface</b></li>
<li><b>EffortJointInterface</b></li>
<li><b>PositionJointInterface</b></li>
<li><b>VelocityJointInterface</b></li>
</ul>

### Configuring and launching the controllers

Once you have defined the transmissions in your URDF file and loaded the Gazebo ros control plugin, it's time to actually begin configuring the controllers.

In order to better understand how you would do this, just follow the next exercise and its explanations!

<p style="background:#EE9023;color:white;">Exercise 2.3</p>
<br>
a) Create a new package named <b><i>my_robot_control</i></b>. Inside this package, create two new folders, one named <b><i>config</i></b> and the other one named <b><i>launch</i></b>.
<br><br>
b) Inside the config folder, create a file named <b><i>my_robot_control.yaml</i></b>. Write the following structure into this file:

So here, you are basically defining two types of controllers: the <b>state controllers</b> (which you saw in the previous chapter that ALWAYS need to be defined) and the <b>effort controllers</b>.

For the position controllers, you need to define the type of plugin that you will use (in this case, it uses the effort_controllers plugin that accepts position commands as input), the name of the joint that it will control, and the pid gains. You'll see 
a little bit more about these pid gains later. 

As you already saw in the previous chapter, the joint_state_controller will allow you to know the state of the joints of your robot at any moment. It will publish this data into the <b>/joint_states</b> topic. For this one, you just need to define the type of plugin that it will use, and the publish rate.

c) Inside the launch folder, create a new file named <i><b>my_robot_control.launch</b></i>. Copy the following contents into that file:

Let's break down this file and explain it in more detail:

In [ ]:
<rosparam file="$(find my_robot_control)/config/my_robot_control.yaml" command="load"/>

What you are doing here is loading the control file you created before into the ROS Parameter Server.

In [ ]:
<node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
    output="screen" ns="/rrbot" args="joint1_position_controller joint2_position_controller joint_state_controller"/>

What you are doing here is starting the two joint position controllers by running a python script that makes a service call to the ros_control controller manager. The service calls tell the controller manager which controllers you want to load. It also loads a third controller that publishes the joint states of all the joints with hardware_interfaces, and advertises the topic on /joint_states.

In [ ]:
<node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"
    respawn="false" output="screen">
    <remap from="/joint_states" to="/rrbot/joint_states" />
  </node>

Finally, here you are starting a robot_state_publisher node that simply listens to /joint_states messages from the joint_state_controller, and then publishes the transforms to /tf. This allows you to see your simulated robot in RVIZ, as well as do other tasks.

d) Launch the file you've just created by executing the following command.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

If you execute a <i>rostopic list</i> in any shell, you will see that the following topics appear.

<img src="img/controller_topics.png" width="500" />

These are the topics that have been created by the launch file you've executed; more specifically, by the controller_spawner node. The <b>command</b> topics will allow you to send goals to your joints, while the <b>joint_states</b> topic will provide you with information about the current state of the joints.

e) Now, you can test your controllers by publishing into their command topics. 

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #2
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Exercise 2.3</p>

<div style="text-align: middle">
<img src="img/joint_pos1.png" width="300" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/joint_pos2.png" width="225" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

<p style="background:#EE9023;color:white;">End of Exercise 2.3</p>

Easy, right? Some things that you didn't understand from the previous chapter are probably now starting to make more sense.

With the previous exercise, you've seen how to create a control package in order to be able to communicate with your joints, and you've launched your controllers within a launch file. You've also interacted with the joints by publishing messages into their 'command' topics. Now, let's have a look at how else you can interact with your joints!

## Interacting with the joints in a graphical way

Until now, you've interacted with the joints of your robot by sending commands to the appropriate topic. But you can also do this by using a very interesting tool that ROS provides: <b>rqt_gui</b>.

In order to see how you can do this, follow the next exercise!

<p style="background:#EE9023;color:white;">Exercise 2.4</p>
<br>
<p style="color:red;"><b>NOTE:</b> Be sure to have your controller running before starting with this exercise.</p>
<br><br>
a) Open the rqt_gui tool by executing the following command.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

b) Hit the icon with a screen in the top-right corner of the IDE window 
<img src="img/font-awesome_desktop.png"/> 
in order to open the Graphic Interface.

In the new window that will open, you will see something like this:

<img src="img/rqt_gui.png" width="600" />

c) On the 'Plugins' menu, select 'Topic' and 'Message Publisher.'

<img src="img/publisher.png" width="400" />

d) In the new window that will appear, choose the topic <i><b>/rrbot/joint1_position_controller/command</b></i>, which is the topic where we can send commands to the joint 1, and click the '+' button.

<img src="img/add.png" width="800" />

e) Now, enable the publisher in the below section by clicking on the check button that appears to the left of the topic name, and set the publish rate to 100.00.

<img src="img/freq_enable.png" width="600" />

f) Next, expand the publisher by clicking the '+' icon to the left of the row. In the 'expression' column, try different radian values and check how the joint moves in the simulation.

g) Now, try to put the next expression in the 'expression' column: <i><b>sin(i/100)</b></i>. Check out what happens in the simulation. You should see the robot moving in the following way:

<img src="img/sin_moves.gif" width="600" />

As you can see, you are making the joint 1 of the robot switch its values by using a sine wave. The letter <i><b>i</b></i> is the variable that rqt uses for time.

h) Next, add a plot display by going to the 'Plugins' menu, 'Visualization,' and 'Plot.' Add the topic <i><b>/rrbot/joint1_position_controller/command</b></i> and click the '+' button.

<div style="text-align: middle">
<img src="img/plot.png" width="300" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/add_plot.png" width="425" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

You should see something like this:

<img src="img/sin_plot.gif" width="600" />

So, here you are visualizing the commands you are sending to the joint 1.

i) Also plot the topic <i><b>/rrbot/joint1_position_controller/state/process_value</b></i>.

<img src="img/process_value.png" width="400" />

This topic shows the actual position of the joint, so you want these two lines to be as equal as possible.

<img src="img/sin_plot2.gif" width="600" />

j) Finally, you are going to open the Dynamic Reconfigure tool, by going to 'Plugins,' 'Configuration,' and 'Dynamic Reconfigure.' This tool allows you to dynamically modify some configuration values of your robot. On the left panel, click on the 'Expand All' button and click on the pid option for the joint 1. Tune the pid values in order to make the two lines as close as possible. 

<div style="text-align: middle">
<img src="img/dynamic_reconfigure.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/dynamic_reconfigure2.png" width="425" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

<p style="background:#EE9023;color:white;">End of Exercise 2.4</p>

<p style="background:red;color:white;">IMPORTANT NOTE</p>
<br>
<b>Be sure to STOP</b> all of the running processes before moving to the next unit. Specifically, the launch file where you launched the controllers.

<p style="background:red;color:white;">IMPORTANT NOTE</p>

And that's all for this unit! Quite interesting, isn't it? Now, if you want to learn how to create your own controller, go to the next unit!